In [14]:
import torch
import torch.nn.functional
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
with open("names.txt", "r") as f:
    words = f.read().splitlines()

words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [16]:
# total words
print("Total names: ", len(words))

print("Minimum name length: ", min(len(w) for w in words))
print("Maximum name length: ", max(len(w) for w in words))

Total names:  32033
Minimum name length:  2
Maximum name length:  15


In [17]:
# making vocabulary mappings
chars = sorted(list(set("".join(words))))
stoi = {c: i+1 for i, c in enumerate(chars)}
stoi["."] = 0
itos = {i: c for c, i in stoi.items()}
print("Stoi: ", stoi)
print("Itos: ", itos)

Stoi:  {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
Itos:  {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


#### build the dataset

In [18]:
block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

In [19]:
X.shape, Y.shape, X.dtype, Y.dtype

(torch.Size([32, 3]), torch.Size([32]), torch.int64, torch.int64)

In [20]:
# In the paper it has 3 words as inputs
# And 17000 words with 30 dimensions
# We only have 27 chars here, let's start with small dimensions for look-up matrix C

In [21]:
C = torch.randn(27, 2)
C.shape

torch.Size([27, 2])

In [22]:
# let' embed a single character first
C[5]

tensor([0.9258, 0.9268])

In [23]:
# Let's embed all
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [24]:
# Now let's make the hidden layer
W1 = torch.randn((6, 100))  # 3*2 inputs and 100 hidden units
b1 = torch.randn(100)